In [1]:
# Ingest the index
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, ServiceContext, load_index_from_storage
import os

from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

import langchain 

langchain.debug = True

# from llama_index import ListIndex
# from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

In [2]:
if os.path.isfile("./storage/index_store.json") :
    print("Found index - loading - this may take a few seconds")
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    # load index
    index = load_index_from_storage(storage_context)
else :
	print("please generate an index first")

print(index)

Found index - loading - this may take a few seconds


In [3]:
tools = [
    Tool(
        name="BITS",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about your class. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

In [4]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")

# memory = GPTIndexChatMemory(
#     index=index,
#     memory_key="chat_history",
#     query_kwargs={"response_mode": "compact"},
#     # return_source returns source nodes instead of querying index
#     return_source=True,
#     # return_messages returns context in message format
#     return_messages=True,
# )

llm = ChatOpenAI(temperature=0, max_tokens=512, model_name="gpt-3.5-turbo") # gpt-4
agent_executor = initialize_agent(tools, llm, agent="conversational-react-description", memory=memory, verbose=True)
# agent_chain = create_llama_chat_agent(tools, llm, memory=memory, verbose=True)

In [5]:
agent_executor.run(input="Hi, My name is Drew. I'm an instructor of BioE80")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Hi, My name is Drew. I'm an instructor of BioE80",
  "chat_history": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Hi, My name is Drew. I'm an instructor of BioE80",
  "chat_history": "",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant 

"\nHi, my name is Drew Endy. I'm an assistant professor of bioengineering at Stanford and I'm an instructor of BioE80. I love teaching this class because I'm excited to see what you can create with bioengineering to make the world a better place. I grew up in Pennsylvania and I was always interested in building things and understanding how things work. That's why I became a bioengineer and why I'm so passionate about this class."

In [6]:
cross_query_str = "Please summarize the BioE80 course"
response = agent_executor.run(input=cross_query_str)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Please summarize the BioE80 course",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: \nHi, my name is Drew Endy. I'm an assistant professor of bioengineering at Stanford and I'm an instructor of BioE80. I love teaching this class because I'm excited to see what you can create with bioengineering to make the world a better place. I grew up in Pennsylvania and I was always interested in building things and understanding how things work. That's why I became a bioengineer and why I'm so passionate about this class."
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Please summarize the BioE80 course",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: \nHi, my name is Drew Endy. I'm an assistant professor of bioengineering at Stanford and I'm an instructor of BioE80. I love teaching this class b

In [7]:
cross_query_str = "Will Bioe80 be offered again next year?"
response = agent_executor.run(input=cross_query_str)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Will Bioe80 be offered again next year?",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: \nHi, my name is Drew Endy. I'm an assistant professor of bioengineering at Stanford and I'm an instructor of BioE80. I love teaching this class because I'm excited to see what you can create with bioengineering to make the world a better place. I grew up in Pennsylvania and I was always interested in building things and understanding how things work. That's why I became a bioengineer and why I'm so passionate about this class.\nHuman: Please summarize the BioE80 course\nAI: \nBioE80 is an introductory course to bioengineering, which focuses on the systematic engineering of living systems to benefit all people and the planet. The course aims to help students learn ways of thinking about engineering living matter, empower them to explore and do bioengineering starting from DNA, and bec

In [8]:
# demonstrate memory
agent_executor.run(input="What's my name?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What's my name?",
  "chat_history": "Human: Hi, My name is Drew. I'm an instructor of BioE80\nAI: \nHi, my name is Drew Endy. I'm an assistant professor of bioengineering at Stanford and I'm an instructor of BioE80. I love teaching this class because I'm excited to see what you can create with bioengineering to make the world a better place. I grew up in Pennsylvania and I was always interested in building things and understanding how things work. That's why I became a bioengineer and why I'm so passionate about this class.\nHuman: Please summarize the BioE80 course\nAI: \nBioE80 is an introductory course to bioengineering, which focuses on the systematic engineering of living systems to benefit all people and the planet. The course aims to help students learn ways of thinking about engineering living matter, empower them to explore and do bioengineering starting from DNA, and become more capable of lear

'\nYour name is Ahmed Best.'

In [9]:
# # reinitialize agent
# memory = ConversationBufferMemory(memory_key="chat_history")
# llm = OpenAI(temperature=0)
# agent_chain = create_llama_chat_agent(
#     toolkit,
#     llm,
#     memory=memory,
# )

In [ ]:
while True:
    text_input = input("User: ")
    response = agent_executor.run(input=text_input)
    print(f"Agent: {response}")